In [1]:
import numpy as np
from scipy.optimize import fsolve
from scipy import interpolate
import matplotlib.pyplot as plt

In [18]:
r = lambda α,k,u: α*k**(α-1)*(1-u)**(1-α)
w = lambda α,k,u: (1-α)*k**(α)*(1-u)**(-α)
u = lambda c: np.log(c)
uprime = lambda c: 1/c

def Euler_solve_k(functions,parameters,states,klevels):
    r,w,gk = functions
    α,β,δ,ur = parameters
    iϵ,k_grid,ϵ_states,ϵ_P = states
    kp,k_bar,kbarp = klevels
    
    ikp = np.where(k_grid==kp)[0]
    nk, nϵ = len(k_grid), len(ϵ_states)
    
    E_Vk = 0
    for iϵp,ϵp in enumerate(ϵ_states):
        E_Vk = E_Vk + ϵ_P[iϵ,iϵp] * 1/(r(α,kbarp,ur)*kbarp + w(α,kbarp,ur)*ϵp +(1-δ)*kp - gk[iϵp*nk+ikp])
    k = 1/(r(α,kbarp,ur)+1-δ) * (1/(β*(r(α,kbarp,ur)+1-δ)*E_Vk) -w(α,kbarp,ur)*ϵ_states[iϵ] + kp)
    return k

def EGM(functions,parameters,states):
    r,w,u = functions
    α,β,δ,ur = parameters
    k_grid,k_bar,ϵ_states,ϵ_P = states
    
    nk,nϵ = len(k_grid),len(ϵ_states)
    gk = np.ones((nϵ,1)).dot(k_grid.reshape(1,nk)).reshape(nϵ*nk,)
    V = np.zeros(nϵ*nk,)
    max_diff = np.inf
    tol = 1e-6
    niter = 1
    
    print('EGM for policy function:')
    
    while max_diff > tol:
        # A vector for expected continuation value. Each capital value stands for k'
        ev = np.zeros((nϵ*nk,))
        for ikp in range(nk):
            idxp = np.arange(ikp,nϵ*nk+ikp,nk)
            for iϵ in range(nϵ):            
                idx = iϵ*nk + ikp 
                ev[idx] = ev[idx] + ϵ_P[iϵ,:].dot(V[idxp])
        
        k_end, V_end, V_new, gk = np.zeros(nϵ*nk,), np.zeros(nϵ*nk,), np.zeros(nϵ*nk,), np.zeros(nϵ*nk,)
        
        for iϵ,ϵ_val in enumerate(ϵ_states):
            grid_range = np.arange(iϵ*nk,iϵ*nk+nk,1)
            
            for ikp,kp in enumerate(k_grid):
                idxp = iϵ*nk + ikp 
                k = Euler_solve_k((r,w,gk),(α,β,δ,ur),(iϵ,k_grid,ϵ_states,ϵ_P),(kp,k_bar,k_bar))
                k_end[idxp] = k
                c = r(α,k_bar,ur)*k + w(α,k_bar,ur)*ϵ_val + (1-δ)*k - kp
                V_end[idxp] = u(c) + β*ev[idxp]
                
            V_end_interp = interpolate.interp1d(k_end[grid_range],V_end[grid_range],fill_value='extrapolate')
            gk_interp = interpolate.interp1d(k_end[grid_range],k_grid,fill_value='extrapolate')
            
            for ik,k in enumerate(k_grid):
                idx = iϵ*nk + ik
                V_new[idx] = V_end_interp(k)
                gk[idx] = max(gk_interp(k),0)
        
        max_diff = max(np.abs(V_new-V))
        V = V_new.copy()
        
        if niter == 1 or niter % 50 == 0:
            print('   Iteration %d: max diff = %1.7f' % (niter,max_diff))
                  
        niter = niter + 1
    return V,gk

def VFI(functions,parameters,states):
    r,w,u = functions
    α,β,δ,ur = parameters
    k_grid,k_bar,ϵ_states,ϵ_P = states
    
    nk,nϵ = len(k_grid),len(ϵ_states)
    gk = np.ones((nϵ,1)).dot(k_grid.reshape(1,nk)).reshape(nϵ*nk,)
    V = np.ones(nϵ*nk,)
    max_diff = np.inf
    tol = 1e-6
    niter = 1
    
    while max_diff > 1e-6:
        # A vector for expected continuation value. Each capital value stands for k'
        ev = np.zeros((nϵ*nk,))
        for ikp in range(nk):
            idxp = np.arange(ikp,nϵ*nk+ikp,nk)
            for iϵ in range(nϵ):            
                idx = iϵ*nk + ikp 
                ev[idx] = ev[idx] + ϵ_P[iϵ,:].dot(V[idxp])
        print(ev)
        V_new = np.zeros(nϵ*nk,)
        
        for iϵ,ϵ_val in enumerate(ϵ_states):
            for ik,k in enumerate(k_grid):
                idx = iϵ*nk + ik
                
                vv = -np.inf
                for ikp,kp in enumerate(k_grid):
                    idxp = iϵ*nk + ikp
                    c = r(α,k_bar,ur)*k + w(α,k_bar,ur)*ϵ_val + (1-δ)*k - kp
                    if c<=0:
                        break
                    else:
                        value = u(c) + β*ev[idxp]
                        if value > vv:
                            vv = value
                            gk[idx] = kp
                            V_new[idx] = value

#                 else:
#                     kp = gk[idx]
#                     idxp = iϵ*nk + np.where(k_grid==kp)[0]
#                     c = r(α,k_bar,ur)*k + w(α,k_bar,ur)*ϵ_val + (1-δ)*k - kp 
#                     V_new[idx] = u(c) + β*ev[idxp]
        
        max_diff = max(np.abs(V_new-V))
        V = V_new.copy()
        print(V)
        if niter == 1 or niter % 50 == 0:
            print('   Iteration %d: max diff = %1.7f' % (niter,max_diff))
                  
        niter = niter + 1
    return V,gk

In [14]:
# Model parameters
β = .96
δ = .06
α = .36
nk = 200
ur_ss = .1      # target unemployment rate at 10%
ϵ_ee = .9       # probability of staying employed in the next period

# Functions for calibration
def calculate_ur_ss(ϵ_uu,ϵ_ee):
    P = np.array([[ϵ_uu,1-ϵ_uu],[1-ϵ_ee,ϵ_ee]])
    mu = np.array([.5,.5])
    diff = np.inf
    
    while diff > 1e-8:
        mu_new = mu.dot(P)
        diff = max(np.abs(mu_new-mu))
        mu = mu_new.copy()
        
    return mu[0]

# Calibrate the probability of staying unemployed to match the unemployment rate target
ϵ_uu = fsolve(lambda ϵ_uu,ϵ_ee: calculate_ur_ss(ϵ_uu,ϵ_ee)-ur_ss,.1,args=(ϵ_ee))
ϵ_states = np.array([0.,1.])
ϵ_P = np.array([[ϵ_uu,1-ϵ_uu],[1-ϵ_ee,ϵ_ee]])
nϵ = len(ϵ_states)

# Generate a grid for capital
k_grid = np.linspace(0.001,40,nk).reshape(nk,)

In [19]:
V,gk = VFI((r,w,u),(α,β,δ,ur_ss),(k_grid,6.490058753339532,ϵ_states,ϵ_P))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

[-9.90351061 -1.38113089 -0.3950593   0.27857601  0.67745163  0.96182261
  1.18297423  1.36941043  1.54909743  1.70136202  1.84504961  1.97646859
  2.09791841  2.21351776  2.31991239  2.42309451  2.51824291  2.6114184
  2.69770066  2.7826399   2.86168846  2.93972995  3.01273293  3.08491328
  3.15277157  3.21990998  3.28332765  3.34608271  3.40562294  3.4645321
  3.52065384  3.57616137  3.62924439  3.68172178  3.73208442  3.78184543
  3.82975746  3.87706953  3.92276178  3.96785468  4.01152633  4.05459895
  4.09642316  4.13764878  4.17777709  4.21730762  4.25587345  4.29384279
  4.33096427  4.36749107  4.40327327  4.43846314  4.47300001  4.50694737
  4.54032322  4.57311283  4.60540362  4.63711185  4.66838629  4.69908219
  4.72940264  4.75914886  4.78857205  4.81742558  4.84600332  4.8740162
  4.90179591  4.92901574  4.95604095  4.98251139  5.00882214  5.03458335
  5.06021657  5.08530557  5.11029539  5.13474636  5.15912438  5.18296899
  5.20676453  5.2300321   5.25327243  5.27600354  5.29

[-9.98251889 -1.46013917 -0.42231814  0.25131717  0.65019279  0.93456376
  1.21441556  1.43701671  1.65430855  1.84602941  2.02316335  2.18264365
  2.3369678   2.48560442  2.62570587  2.75897178  2.88702044  3.01258459
  3.13186606  3.24667577  3.35864694  3.46813641  3.57146046  3.6734747
  3.77295828  3.87054002  3.96291731  4.05471313  4.14421668  4.23256682
  4.31644651  4.39978876  4.48113308  4.56191163  4.6388449   4.71522664
  4.78986841  4.86419572  4.93531047  5.00583829  5.0750278   5.14364088
  5.20979455  5.2753172   5.33980426  5.40351927  5.46538395  5.52657295
  5.58696117  5.64643102  5.70454575  5.76194361  5.81872648  5.87448166
  5.92928667  5.9833388   6.03692511  6.0894025   6.14126277  6.19233973
  6.24307208  6.29263575  6.34186558  6.39027124  6.43843966  6.48539621
  6.53226088  6.57826647  6.62410716  6.66890673  6.71343729  6.7573176
  6.80099834  6.84383652  6.88625134  6.92819484  6.9699102   7.01095379
  7.05144483  7.09161589  7.13153525  7.17093036  7.2

[-9.89855344 -1.37617372 -0.31207245  0.36156286  0.76043848  1.05319674
  1.3343483   1.57544899  1.79274084  1.98767722  2.16956463  2.34782675
  2.52247935  2.6830415   2.83803159  2.98573517  3.13176248  3.27401785
  3.40919614  3.5407315   3.66690575  3.79185966  3.91608558  4.0367508
  4.15229971  4.26568065  4.37527883  4.48384286  4.59124546  4.69708689
  4.79893745  4.89919389  4.99639405  5.09243526  5.18797999  5.282442
  5.37390173  5.46396523  5.55130006  5.63791512  5.72372958  5.80914996
  5.89232987  5.9741149   6.05352929  6.1324789   6.21079325  6.28835028
  6.36467395  6.43958852  6.51268331  6.58514298  6.65712497  6.7280757
  6.7986193   6.86774105  6.93548955  7.00259502  7.06905835  7.13482338
  7.20003468  7.26420133  7.32737452  7.38985362  7.45158896  7.51303914
  7.57351633  7.63339409  7.69258669  7.75107543  7.80889346  7.86635371
  7.9227378   7.9788661   8.03456102  8.08954905  8.14400101  8.19787523
  8.25086797  8.30356665  8.35610273  8.40798883  8.459

[-9.77534855 -1.25296883 -0.18646718  0.48716813  0.88604375  1.18033644
  1.46148799  1.71097691  1.92826875  2.13785473  2.32892167  2.50956684
  2.68466755  2.85810543  3.02417003  3.18149361  3.33294846  3.48137084
  3.62570899  3.76861706  3.90985994  4.04647907  4.17777016  4.30699133
  4.43308923  4.55806198  4.6798639   4.80067925  4.91937687  5.0356182
  5.14838332  5.25939982  5.36906296  5.47748546  5.58284376  5.68775606
  5.79172516  5.89393082  5.99335663  6.09156898  6.1887198   6.28458471
  6.37834911  6.47109408  6.56323856  6.65471496  6.74376068  6.83204848
  6.91946572  7.00539206  7.09031146  7.17392033  7.2566549   7.33880153
  7.41955303  7.49979929  7.57933158  7.65741247  7.73487867  7.8112407
  7.88631666  7.96064397  8.0347436   8.10825452  8.18119146  8.25306287
  8.32394282  8.39422916  8.46315122  8.53166619  8.59977001  8.6672711
  8.73455013  8.80111009  8.86649707  8.93155539  8.99567117  9.05914334
  9.12213856  9.18472233  9.24690095  9.30872457  9.36

[-9.64921243 -1.12683271 -0.05226114  0.62137417  1.02024979  1.31967165
  1.60082321  1.85273119  2.07002303  2.28151522  2.4779651   2.66424389
  2.84407856  3.01751644  3.18797676  3.35445812  3.51311937  3.66799758
  3.81790415  3.96612846  4.11068893  4.25256551  4.39340921  4.53169889
  4.66633916  4.79869129  4.92816766  5.05638079  5.18155496  5.30448341
  5.42515219  5.54442572  5.66276415  5.77954587  5.89447216  6.00765099
  6.11930049  6.22979979  6.33765865  6.44444353  6.54926825  6.65310359
  6.75502266  6.85642263  6.95722471  7.05683717  7.15527347  7.25250608
  7.34800951  7.44250511  7.53584373  7.62760857  7.71841857  7.80841861
  7.89738509  7.98593692  8.07411951  8.16106778  8.24703314  8.33189993
  8.41581348  8.49828865  8.58020034  8.66160118  8.74231566  8.82170223
  8.90069678  8.97922883  9.05734967  9.13452971  9.21057678  9.28594943
  9.36076811  9.43488736  9.50840562  9.58123928  9.65308974  9.72441048
  9.79543368  9.8659838   9.93569434 10.00497939 10

KeyboardInterrupt: 